# Information Visualization I 
## School of Information, University of Michigan

## Week 1: 
- Domain identification vs Abstract Task extraction
- Pandas Review 

## Assignment Overview
### The objectives for this week are for you to:

- Review, reflect, and apply the concepts of Domain Tasks and Abstract Tasks. Specifically, given a real context, identify the expert's goals and then abstract the visualization tasks. 

!["Drawing"](assets/domain-abstraction.png)



- Review and evaluate the domain of [Pandas](https://pandas.pydata.org/) as a tool for reading, manipulating, and analyzing datasets in Python.

### The total score of this assignment will be 100 points consisting of:
- Case study reflection: Car congestion and crash rates (20 points)
- Pandas programming exercise (80 points)


### Resources:
- We're going to be recreating parts of this article by [CMAP](https://www.cmap.illinois.gov/) available [online](https://www.cmap.illinois.gov/updates/all/-/asset_publisher/UIMfSLnFfMB6/content/crash-scans-show-relationship-between-congestion-and-crash-rates) (CMAP, 2016)  
- We'll need the datasets from the city of Chicago. We have downloaded a subset to the local folder [/assets](assets/)
    - If you're curious, the original dataset can be found on [Chicago Data Portal](https://data.cityofchicago.org/)
        - [Chicago Traffic Tracker - Historical Congestion Estimates by Segment - 2011-2018](https://data.cityofchicago.org/Transportation/Chicago-Traffic-Tracker-Historical-Congestion-Esti/77hq-huss)
        - [Traffic Crashes - Crashes](https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if)
- Pandas
    - This assignment is partially a warm-up/reminder of how to use Pandas. We've also created an optional lab for you (see Coursera) if you need more help remembering how to do things in Pandas.
- Altair
    - We will use a python library called [Altair](https://altair-viz.github.io/) for the visualizations. Don't worry about understanding this code. You will only need to prepare the data for the visualization in Pandas. If you do it correctly, our code will produce the visualization for you.
    - If you're interested, we made a short [7-minute video](https://www.youtube.com/watch?v=Tg41r3lAYoQ) explaining the very basics of how Grammar of Graphics/Altair works and why we need to transform the data as we do.
    
    
### Important notes:
1) Grading for this assignment is entirely done by a human grader. They will be running tests on the functions we ask you to create. This means there is no autograding (submitting through the autograder will result in an error). You are expected to test and validate your own code. 

2) Keep your notebooks clean and readable. If your code is highly messy or inefficient you will get a deduction.

3) Pay attention to the inputs and return types of your functions. Sometimes things will look right but fail later if you return the wrong kind of object (e.g., Array instead of Series). *Do not* hard-code variables into your functions (however, if we say it's a dataframe of 'type' X being passed in, it's ok to assume that the standard columns of X exist). *Do not* modify our function definitions.

4) Follow the instructions for submission on Coursera. You will be providing us a generated link to a read-only version of your notebook and a PDF. When turning in your PDF, please use the File -> Print -> Save as PDF option ***from your browser***. Do ***not*** use the File->Download as->PDF option. Complete instructions for this are under Resources in the Coursera page for this class. If you're having trouble with printing, take a look at [this video](https://youtu.be/PiO-K7AoWjk).

## Part 1. Domain identification vs Abstract Task extraction (20 points)
Read the following article by CMAP [Crash scans show the relationship between congestion and crash rates](https://www.cmap.illinois.gov/updates/all/-/asset_publisher/UIMfSLnFfMB6/content/crash-scans-show-relationship-between-congestion-and-crash-rates) and answer the following questions. Think of this as the output report produced by the analyst.

Remember: Domain tasks are questions an analyst might want to answer and/or they might be insights (answers) the analyst wants to communicate to someone else. For example, a retail analyst might want to know: how many fruit did we sell? or what’s the relationship between temperature and fruits rotting? A learning analyst would have the domain task: how often do students pass the class? or how does study time correlate with grade? An advertising analyst would ask: how many people clicked on an ad? or what’s the relationship between time of day and click through rate? 

Abstract tasks are generic: What’s the sum of a quantitative variable? or what’s the correlation between two variables? Notice we gave two examples for each analyst type and these roughly map to the two abstract questions. You should not use domain language (e.g., accidents) when describing abstract tasks. 


### 1.1 Briefly describe who you think performed this analysis. What is their expertise? What is their goal for the article? Give 3 examples of domain tasks featured in the article. (10 points)

_1.1 Answer_

### 1.2  For each domain task describe the abstract task (10 points)

_1.2 Answer_

## Part 2. Pandas programming exercise (80 points)
We have provided some code to create visualizations based on these two datasets:
1. [Historic Congestion](assets/Pulaski.small.csv.gz) 
2. [Traffic Crashes](assets/Traffic.Crashes.csv.gz)

Complete each assignment function and run each cell to generate the final visualizations


In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
# enable correct rendering
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [3]:
# uses intermediate json files to speed things up
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

### PART A: Historic Congestion ( 55 points)
For parts 2.1 to 2.5 we will use the Historic Congestion dataset. This dataset contains measures of speed for different segments. For this subsample, the available measures are limited to traffic on Pulaski Road in 2018.

### 2.1 Read and resample (15 points)
Complete the `read_csv` and `get_group_first_row` functions.
Since our dataset is large we want to only grab one measurement per hour for each segment. To do this, we will resample by grouping based on some columns (e.g., month, day, hour for each segment) and then picking out the first measurement from that group. We're going to write the sampling function to be generic. Complete the `get_group_first_row` function to achieve this. Note that the file we are loading is compressed--depending on how you load the file, this may or may not make a difference ([you'll want to look at the API documents](https://pandas.pydata.org/pandas-docs/stable/reference/index.html)).

In [4]:
def read_csv(filename):
    return pd.read_csv(filename,compression='gzip')

In [107]:
# Save the congestion dataframe on hist_con
hist_con = read_csv('assets/Pulaski.small.csv.gz',)

# let's look at the raw data and check that it's ok
print(hist_con.shape)
assert hist_con.shape == (3195450, 10)
assert list(hist_con.columns) == ['TIME','SEGMENT_ID','SPEED','STREET','DIRECTION','FROM_STREET','TO_STREET',
                                  'HOUR','DAY_OF_WEEK','MONTH']

TypeError: read_csv() got an unexpected keyword argument 'parse_dates'

In [7]:
# let's get rid of -1's on SPEED because that's missing data
hist_con = hist_con[hist_con.SPEED>-1]

In [33]:
from itertools import groupby


def get_group_first_row(df, grouping_columns):
    """Group rows using the grouping_columns argument and return the first row belonging to each group
    (you can look at first() for reference). We'll write this function to be more general in case
    we want to use it for a different resample.
    return a dataframe without a hierarchical index (important: return with default index)
    
    See the example link below if you want a better sense of what this should return
    """
   

    return df.groupby(grouping_columns).first().reset_index()
  

Index(['MONTH', 'DAY_OF_WEEK', 'HOUR', 'SEGMENT_ID', 'TIME', 'SPEED', 'STREET',
       'DIRECTION', 'FROM_STREET', 'TO_STREET'],
      dtype='object')

In [41]:
# test your code, we want segment_rows to be resampled version of hist_con where we've grouped by the
# properties month, day_of_week, hour, and segment_id and returned the first measure of each group
segment_rows = get_group_first_row(hist_con, ['MONTH','DAY_OF_WEEK', 'HOUR', 'SEGMENT_ID'])

assert type(segment_rows) == pd.DataFrame
assert len(segment_rows.columns) == 10
assert list(segment_rows.columns[:4]) == ['MONTH','DAY_OF_WEEK', 'HOUR', 'SEGMENT_ID']

segment_rows.sample(5)

,MONTH,DAY_OF_WEEK,HOUR,SEGMENT_ID,TIME,SPEED,STREET,DIRECTION,FROM_STREET,TO_STREET
99536,11,2,11,61,11/26/2018 11:50:22 AM,29,Pulaski,SB,99th,103rd
48867,7,1,0,53,07/29/2018 12:50:21 AM,27,Pulaski,NB,Irving Park,Elston
57906,7,6,7,22,07/27/2018 07:50:31 AM,26,Pulaski,NB,99th,95th
103568,11,4,18,31,11/28/2018 06:30:58 PM,15,Pulaski,NB,59th,55th
24447,4,7,21,35,04/21/2018 09:30:55 PM,36,Pulaski,NB,43rd,I-55 Expy


The table should look something like [this](assets/segment_rows.png). 

***Note** When we show examples like this, we are sampling (e.g., ```segment_rows.sample(5)```) so your table may look different.

If you want to build your own tests from our example tables, you can create an assert test for one of the rows and make sure the values match what you expect. For example we see that the row id 68592 in the example is for 8/27/2018 at 7:50:28 AM. So we could write the test:

```assert segment_rows.loc[62986].TIME == '08/27/2018 07:50:28 AM'```

If this assertion failed, you'd get an error message.

Now let's do something a little bit interesting with this. We should now be able to test a theory that traffic speeds vary by hour of day. We're going to create a scatter plot showing hour on the x-axis and speed on the y-axis. We're going to sample only one hour per segment to keep things simple. So for each segment (we have 78 of them) we're going to find the first speed measure for 12am, 1am, 2am, etc. The result will be roughly 1844 points (plus or minus, we have some missing data). On top of that, we will add a line for the mean speed for each hour. To plot this, we need our data to look roughly like this:

!["table 2.1"](assets/table_2.1.png)

This will allow the encoding system to read row by row, and create a point for each where the X is the hour and Y is the speed. If everything works, you'll see:

!["vis 2.1"](assets/vis_2.1.png)

Notice the dip in speeds around morning and afternoon rush hours. 

In [35]:
def create_mean_speed_vis(indf):
    # input: indf -- the input frame (in style of hist_con above)

    # sub-sample for hour/segment
    srows = get_group_first_row(indf, ['HOUR', 'SEGMENT_ID'])

    # grab the only columns we care about (strictly speaking, we only need HOUR and SPEED)
    srows = srows[['HOUR','SEGMENT_ID','SPEED']]

    # create the scatter plot using this data
    distr = alt.Chart(srows).mark_circle().encode(
        x='HOUR:Q',  # x is the HOUR
        y='SPEED:Q'  # y is the speed
    )

    # create the line chart on top, we could calculate the means in either Pandas or Altair
    mean = distr.mark_line(color='red').encode(
        # this "extends" distr, so x is still encoding HOUR
        y='mean(SPEED):Q' # y should now encode the mean of SPEED (at each hour)
    )

    # combine the scatter plot and line chart
    return distr+mean

create_mean_speed_vis(hist_con)

alt.LayerChart(...)

### 2.2 Basic Bar Chart Visualization (10 points)
We want to create a bar chart visualization for the *average speed* of each segment (across all the samples). Our encoder is going to want the data so we that we have one row per segment, with a segment id column (we'll use this for the X placement of the bars) and the average speed (we'll use this for the length of the bar). So something like this:

!["table 2.2"](assets/table_2.2.png)

To do this, we're going to want to group by each segment and calculate the average speed on each. Complete this code on the `average_speed_per_segment` function. Make sure your function returns a ***series***.

In [53]:
def average_speed_per_segment(df):
   
   return df.groupby(['SEGMENT_ID'])['SPEED'].mean()

In [59]:
# reset to a "clean" segment_rows
segment_rows = get_group_first_row(hist_con, ['MONTH','DAY_OF_WEEK', 'HOUR', 'SEGMENT_ID'])

# calculate the average speed per segment
average_speed = average_speed_per_segment(segment_rows)

assert type(average_speed) == pd.Series

# check what's in average_speed
average_speed

SEGMENT_ID
19    29.573699
20    25.585315
21    28.564897
22    27.116505
23    27.698651
        ...    
93    21.467613
94    20.209259
95    20.987692
96    29.769136
97    25.357804
Name: SPEED, Length: 78, dtype: float64

If you got things right, the **series** should look something like [this](assets/average_speed.png). You might want to write a test to make sure you are returning the expected type. For example:

```assert type(average_speed) == pd.core.series.Series```

In [55]:
# make a dataframe from the average_speed
def get_average_speed_df(indf):
    # input: indf the input data frame (like hist_con)
    # reset segment rows
    segment_rows = get_group_first_row(indf, ['MONTH','DAY_OF_WEEK', 'HOUR', 'SEGMENT_ID'])
    
    # calculate the average speed
    average_speed = average_speed_per_segment(segment_rows)
    # create the data frame
    asdf = pd.DataFrame(average_speed).reset_index()
    #return the frame
    return asdf


In [61]:
# see what's inside
average_speed_df = get_average_speed_df(hist_con)

assert type(average_speed_df) == pd.DataFrame
assert len(average_speed_df.columns) == 2
assert average_speed_df.SPEED.dtype == float

# print a sample
average_speed_df.sample(5)

,SEGMENT_ID,SPEED
14,33,23.899320
54,74,23.404612
36,55,20.819188
45,65,23.990043
21,40,21.792488


Ok, now we can build our visualization. If your code is correct, you should seem something like:
    
!["vis 2.2"](assets/vis_2.2.png)

In [58]:
# let's generate the visualization

def create_average_speed_per_segment_vis(visdf):
    # visdf: frame to visualize
    
    # create a chart
    base = alt.Chart(visdf)

    # we're going to "encode" the variables, more on this next assignment
    encoding = base.encode(
        x= alt.X(                # encode SEGMENT_ID as a quantiative variable on the X axis
                'SEGMENT_ID:Q',
                title='Segment ID',
                scale=alt.Scale(zero=False)   # we don't need to start at 0
        ),
        y=alt.Y(
                'sum(SPEED):Q',   # encode the sum of speed for the segment as a quantitative variable on Y
                title='Speed Average MPH'
        ),
    )

    # we're going to use a bar chart and set various parameters (like bar size and title) to make it readable
    return encoding.mark_bar(size=7).properties(title='Average Speed per Segment',height=300, width=900)

create_average_speed_per_segment_vis(average_speed_df)

alt.Chart(...)

### 2.3 Create a basic pivot table (10 points)
For the next visualization, we need a more complex transformation that will allow us to see the average speed for each month. We're going to use a heatmap style calendar visualization (think GitHub) check-in history. Our encoder is going to make a square for each segment/month. The segment id will tell us where on the x-axis to put the square and the month value will tell us where on the y-axis. We will also want the mean speed as a column (for that month/segment) which we'll encode using color. What we're working towards is a dataframe that looks something like:

!["table 2.3"](assets/table_2.3.png)

We're going to do part of this for you. First, we need you to use a pivot table to get us part way there.
For the pivot table we want a table where the index is the month, and each column is a segment id. We will put the average speed in the cells.

Complete the `create_pivot_table` function for this. The table you output should look something like [this](assets/pivot_table.png)

In [74]:
from calendar import month


def create_pivot_table (df):
    """return a pivot table where:
    each row i is a month
    each column j is a segment id
    each cell value is the average speed for the month i in the segment j
    """
    return pd.pivot_table(df, values=['SPEED'], index=['MONTH'], columns=['SEGMENT_ID'], aggfunc=np.mean)

In [75]:
# go back to our original sample for segment_rows
segment_rows = get_group_first_row(hist_con, ['MONTH','DAY_OF_WEEK', 'HOUR', 'SEGMENT_ID'])

In [76]:
# run the code and see what's in the table
pivot_table = create_pivot_table(segment_rows)
pivot_table

SPEED                                                         \
SEGMENT_ID         19         20         21         22         23         24   
MONTH                                                                          
2           28.166667  23.142857  24.571429  27.285714  24.142857  29.000000   
3           29.306569  26.223776  27.649635  26.174242  27.553030  30.560606   
4           29.753623  26.097902  28.428571  26.825758  28.128788  31.393939   
5           30.708029  26.302817  29.007576  28.068182  28.522727  31.545455   
6           29.833333  26.517483  28.823529  27.096296  26.834586  30.208955   
7           30.153285  25.127660  28.848485  27.606061  26.946970  31.545455   
8           30.755396  25.867133  28.874074  27.507463  28.240602  32.127820   
9           30.136691  25.545455  29.207143  27.274074  27.849624  32.255639   
10          29.357664  24.723404  29.389706  28.203008  27.954887  30.766917   
11          28.919708  24.765957  28.231343  26.609023  27.586466  31.030075   
12          26.884892  24.776224  27.380597  25.798507  27.559701  29.634328   

                                                        ...             \
SEGMENT_ID         25         26         27         28  ...         88   
MONTH                                                   ...              
2           24.000000  28.714286  30.142857  24.285714  ...  20.714286   
3           22.421053  23.923077  28.460000  22.181208  ...  21.083333   
4           23.631579  25.517007  28.401316  21.456954  ...  20.446429   
5           23.661654  25.442177  29.615894  22.613333  ...  20.690476   
6           22.984962  25.282759  29.250000  21.647059  ...  20.541667   
7           23.431818  24.958042  30.373333  21.738255  ...  20.589286   
8           23.295455  25.951049  28.335526  21.922581  ...  20.410714   
9           23.757576  25.258741  28.453333  21.111842  ...  21.500000   
10          24.992424  25.770833  27.966887  22.857143  ...  20.750000   
11          24.753731  26.137931  27.423841  23.176471  ...  21.648810   
12          24.786765  24.573427  28.390728  23.389610  ...  22.226190   

                                                                              \
SEGMENT_ID         89         90         91         92         93         94   
MONTH                                                                          
2           18.428571  22.428571  21.285714  20.857143  22.142857  20.714286   
3           19.101190  22.321429  20.214286  23.297619  20.882716  20.530864   
4           18.005952  22.696429  19.952381  23.416667  22.391304  19.341615   
5           18.333333  23.244048  20.327381  23.267857  21.714286  19.496894   
6           18.071429  23.184524  19.494048  23.517857  21.870370  19.708075   
7           17.833333  22.886905  19.767857  24.273810  21.130435  20.248447   
8           17.892857  23.291667  19.750000  23.535714  21.732919  19.782609   
9           18.946429  23.416667  19.613095  23.226190  20.583851  19.745342   
10          19.291667  23.714286  22.071429  23.261905  21.167702  20.062112   
11          20.988095  24.416667  20.803571  22.535714  20.080745  20.540373   
12          20.511905  23.148810  23.095238  24.625000  23.073620  22.582822   

                                             
SEGMENT_ID         95         96         97  
MONTH                                        
2           21.714286  30.000000  25.714286  
3           20.159509  29.283951  25.246914  
4           20.709877  29.701863  25.385093  
5           21.099379  29.757764  25.833333  
6           21.074534  30.006211  25.546584  
7           21.341463  29.828221  24.546012  
8           21.839506  29.981366  25.714286  
9           20.956522  29.080745  24.819876  
10          20.850932  29.708075  25.242236  
11          20.453416  29.993789  25.093168  
12          21.358025  30.341615  26.142857  

[11 rows x 78 columns]

As before, we can write a "test" based on this example.  For example, [here](assets/pivot_table.png) we see that in March (Month 3) segment 21 had a value of ~27.65, so we could write the test:

```assert round(pivot_table.loc[3,21],3) == 27.65```

In [95]:
# add your tests here
assert len(pivot_table.index.values) < 13
assert pivot_table.columns.dtype == object



In [96]:
# we're going to implement a transformation to put the pivot table into a 'long form' because it
# is easier to specify the visualization.
def make_long_form(sourceTable):
    # sourceTable: the original table to modify
    hm_pivot_table = sourceTable.copy().unstack().reset_index()
    hm_pivot_table['SPEED'] = hm_pivot_table[0]
    hm_pivot_table.drop(0,axis=1,inplace=True)
    return(hm_pivot_table)

In [97]:
# you can see what's inside the long form
longformASSM = make_long_form(pivot_table)
longformASSM.sample(5)

,level_0,SEGMENT_ID,MONTH,SPEED
673,SPEED,81,4,23.452381
479,SPEED,63,8,28.884892
711,SPEED,84,9,23.267857
641,SPEED,78,5,20.521472
144,SPEED,32,3,20.335664


In [98]:
# create the visualization. We're going to use rectangles (a heat map of sorts). We'll use the segment_id to
# figure out the horizontal placement (x), the month as the vertical (y) and use color to encode the speed.
def create_speed_month_segment_vis(visframe):
    # visframe: the frame to visualize
    
    # using rectangles
    encoding = alt.Chart(visframe).mark_rect().encode(
        x='SEGMENT_ID:O',   # segments on the x axis, ordinal encoding so ordered
        y='MONTH:O',        # month, ordinal encoding so ordered
        color='SPEED:Q'     # color based on speed, quantitative encoding
    )

    # adjust title, height, width and return
    return encoding.properties(title='Average Speed per Segment per Month',height=300, width=800)

create_speed_month_segment_vis(longformASSM)

alt.Chart(...)

### 2.4 Sorting, Transforming, and Filtering (20 points)
Without telling you too much about the visualization we want to create next (that's part of the bonus below), we need to get the data into a form we can use. In the end, we'll want something roughly like:

!["table 2.4"](assets/table_2.4.png)

To do this:
- We're going to need to sort the dataframe by one or more columns (this is the `sort_by_col` function). 
- We'll want to create a derivative column that is the time of the measurement rounded to the nearest hour (`time_to_hours`)
- We need to "facet" the data into groups to generate different visualizations. 
- We need a function that selects part of the dataframe that matches a specific characteristic (`filter_orientation`)

In [99]:
def sort_by_col(df, sorting_columns):
    """Sort the rows of df by the columns (sorting_columns). Use default sort order (ascending).
    return the sorted dataframe
    """
    return df.sort_values(by=sorting_columns)

In [100]:
# test it out
segment_rows = sort_by_col(segment_rows, ['SEGMENT_ID'])
segment_rows.sample(5)

,MONTH,DAY_OF_WEEK,HOUR,SEGMENT_ID,TIME,SPEED,STREET,DIRECTION,FROM_STREET,TO_STREET
119325,12,6,20,46,12/28/2018 08:40:09 PM,18,Pulaski,NB,Division,Grand
52732,7,3,8,46,07/31/2018 08:20:43 AM,20,Pulaski,NB,Division,Grand
6134,3,4,7,76,03/28/2018 07:50:31 AM,27,Pulaski,SB,31st,I-55 Expy
42073,6,4,4,26,06/27/2018 04:50:17 AM,30,Pulaski,NB,83rd,79th
38425,6,2,0,97,06/25/2018 12:50:19 AM,26,Pulaski,SB,Peterson,Bryn Mawr


In [108]:
def time_to_hours(df):
    """ Add a column (called TIME_HOURS) based on the data in the TIME column and rounded to
    the value to the nearest hour.  For example, if the original TIME row said: 
    ‘02/28/2018 05:40:00 PM’ we want ‘2018-02-28 18:00:00’  
    (the change is that 5:40pm was rounded up to 6:00pm and the TIME_HOUR column is 
    actually a proper datetime and not a string). The column should be a datetime type.
    You can use standard time rounding functions, but if you implement this yourself round up to break ties.
    """
    df['TIME'] = pd.to_datetime(df['TIME'])
    df['TIME_HOURS'] = [row.round(freq='H') for row in df['TIME']]
    return df

In [109]:
# we can test this out
segment_rows = time_to_hours(segment_rows)
segment_rows.sample(5)

,MONTH,DAY_OF_WEEK,HOUR,SEGMENT_ID,TIME,SPEED,STREET,DIRECTION,FROM_STREET,TO_STREET,TIME_HOURS
75282,9,2,9,93,2018-09-24 09:40:09,10,Pulaski,SB,Elston,Irving Park,2018-09-24 10:00:00
54366,7,4,6,81,2018-07-25 06:50:26,31,Pulaski,SB,Van Buren,Roosevelt,2018-07-25 07:00:00
51508,7,2,15,30,2018-07-30 15:50:25,29,Pulaski,NB,63rd,59th,2018-07-30 16:00:00
62966,8,2,7,30,2018-08-27 07:50:27,29,Pulaski,NB,63rd,59th,2018-08-27 08:00:00
44899,6,5,17,81,2018-06-28 17:40:12,15,Pulaski,SB,Van Buren,Roosevelt,2018-06-28 18:00:00


In [118]:
def filter_orientation(df, traffic_orientation):
    """ Filter the rows according to the traffic orientation
    return a df that is a subset of the original with the desired orientation
    df: original traffic data frame
    traffic_orientation: a string, one of "SB" or "NB"
    """
    return df[df['DIRECTION'] == traffic_orientation]

In [119]:
# let's filter down to a south bound and north bound table
sb = filter_orientation(segment_rows, 'SB')
nb = filter_orientation(segment_rows, 'NB')

The sb table should look like [this](assets/sb.png)

In [112]:
# let's look at a sample. You might want to build some assert tests here
sb.sample(5)

,MONTH,DAY_OF_WEEK,HOUR,SEGMENT_ID,TIME,SPEED,STREET,DIRECTION,FROM_STREET,TO_STREET,TIME_HOURS
52624,7,3,6,95,2018-07-31 06:50:24,20,Pulaski,SB,Foster,Lawrence,2018-07-31 07:00:00
60464,7,7,17,93,2018-07-28 17:50:20,18,Pulaski,SB,Elston,Irving Park,2018-07-28 18:00:00
108135,11,7,9,91,2018-11-24 09:50:24,23,Pulaski,SB,Milwaukee,Diversey,2018-11-24 10:00:00
7868,3,5,7,59,2018-03-29 07:50:23,29,Pulaski,SB,107th,111th,2018-03-29 08:00:00
59819,7,7,9,72,2018-07-28 09:40:09,23,Pulaski,SB,Archer,55th,2018-07-28 10:00:00


In [113]:
# let's put it all together to generate our table
def get_sbnb(indf):
    # input: indf, a hist_con shaped data frame
    
    # go back to our original sample for segment_rows
    segment_rows = get_group_first_row(indf, ['MONTH','DAY_OF_WEEK', 'HOUR', 'SEGMENT_ID'])
    
    # use our new functions
    segment_rows = sort_by_col(segment_rows, ['SEGMENT_ID'])
    segment_rows = time_to_hours(segment_rows)
    sb = filter_orientation(segment_rows, 'SB')
    nb = filter_orientation(segment_rows, 'NB')

    # now append the columns and just select the sub columns we care about
    sbnb = sb.append(nb)[['DIRECTION','FROM_STREET','TIME_HOURS','SPEED']]
    return(sbnb)

In [114]:
# let's see what's inside
sbnb = get_sbnb(hist_con)
sbnb.sample(5)

/var/folders/76/sj1qf7k94mb1p4hpvsb0mv880000gn/T/ipykernel_91233/2906947039.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sbnb = sb.append(nb)[['DIRECTION','FROM_STREET','TIME_HOURS','SPEED']]


,DIRECTION,FROM_STREET,TIME_HOURS,SPEED
54006,SB,Van Buren,2018-07-25 01:00:00,36
102398,NB,Division,2018-11-28 02:00:00,27
112630,SB,26th,2018-12-25 01:00:00,23
8529,SB,Bryn Mawr,2018-03-29 16:00:00,33
10432,SB,Foster,2018-03-30 18:00:00,20


In [115]:
# create the visualization, but it's your bonus (2.5) to describe what's going on
def create_speed_direction_vis(visdf):
    alt.data_transformers.disable_max_rows()
    return alt.Chart(visdf).mark_rect().encode(
        x='month(TIME_HOURS):T',
        y='FROM_STREET:N',
        color='mean(SPEED):Q',
        facet='DIRECTION:N'
    ).properties(
        width=300,
        height=400
    )

create_speed_direction_vis(sbnb)

alt.Chart(...)

### 2.5 (Bonus)  Traffic heatmap visualization (up to 2 points)
Looking at the visualization above (the one showing Northbound versus Southbound facets), what domain/abstract tasks are fulfilled by this visualization? List at least one domain task and the corresponding abstract task.

_2.5 Answer_

### PART B: Crashes (25 points)
For parts 2.6 and 2.7 we will use the Crashes dataset. This dataset contains crash entries recording the time of the accident, the street, and the street number where the accident occurred. You will work with accidents recorded on Pulaski Road


In [121]:
# load the crash data
crashes = read_csv('assets/Traffic.Crashes.csv.gz')

# just grab the pulaski road data
crashes_pulaski = crashes[crashes.STREET_NAME == 'PULASKI RD']

crashes_pulaski.head()

,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
6,JC100005,NaN,12/31/2018 11:45:00 PM,25,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,UNKNOWN,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,...,0.0,0.0,3.0,0.0,23,2,12,41.851521,-87.724905,POINT (-87.72490478675 41.851521442331)
51,JB574321,NaN,12/31/2018 09:20:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,...,0.0,0.0,2.0,0.0,21,2,12,41.807977,-87.723443,POINT (-87.723442855227 41.807977202659)
126,JB574112,NaN,12/31/2018 05:30:00 PM,30,NO CONTROLS,NO CONTROLS,RAIN,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,PARKING LOT,...,0.0,0.0,2.0,0.0,17,2,12,41.799320,-87.723191,POINT (-87.72319071101 41.799320431921)
133,JC111739,NaN,12/31/2018 05:30:00 PM,30,NO CONTROLS,NO CONTROLS,RAIN,DUSK,REAR END,NOT DIVIDED,...,0.0,0.0,4.0,0.0,17,2,12,41.822564,-87.724187,POINT (-87.724187479133 41.822563835982)
240,JB573809,NaN,12/31/2018 01:30:00 PM,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,TURNING,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0,0.0,6.0,0.0,13,2,12,41.796759,-87.723116,POINT (-87.723115903434 41.79675909896)


### 2.6 Calculate summary statistics for grouped streets (15 points)

We want to get a few summary visualizations like where crashes are happening on Pulaski Road (by which house number). We're going to bin the records by house number to start. Think of bins as vaguely representing "street blocks" (it's obviously not quite right).

- Group the streets every 300 units (street numbers). Hint: You can use the `pd.cut` function

The second visualization will tell us around which houses accidents are happening.

- Calculate the number of accidents (count rows) and the total of injuries (sum injuries total) for each of these 300-chunk road segments. Do this *for each direction*.

Complete ```bin_crashes``` and ```calculate_group_aggregates``` functions for this.


In [ ]:
def bin_crashes(df):
    """ Assign each crash instance a category (bin) every 300 house number units starting from 0
    Return a new dataframe with a column called BIN where each value is the start of the bin
    i.e. 0 is the label for records with street number n, where 1 <= n <= 300
    300 is the label for records with n at 301 <= n <= 600, and so on.
    """


In [ ]:
binned_df = bin_crashes(crashes_pulaski)

# sample the values to see what's in your new DF (we only care about street no and bin)
binned_df.sample(5)[['STREET_NO','BIN']]

A sample of the relevant columns from the table would look something like [this](assets/binned_df.png). We can also create a histogram of street numbers to see which are the most prevalent. It should look something like [this](assets/street_no.png).

In [ ]:
def create_street_histogram_vis(visf):
    # create this vis
    return alt.Chart(visf).mark_bar().encode(
        alt.X('BIN'),
        alt.Y('count()')
    )

create_street_histogram_vis(bin_crashes(crashes_pulaski))

In [ ]:
def calculate_group_aggregates(df):
    """ 
    There are *accidents* and *injuries* (could be 0 people got hurt, could be more). 
    There’s one row per accident at the moment, so we want to know how many accidents 
    happened in each BIN/STREET_DIRECTION (this will be the count) and how many injuries (which will be the sum). 
    
    Return a df with the count of accidents in a column named 'ACCIDENT_COUNT' (how many accidents happened in each 
    bin (the count) and how many injuries (the sum) in a column named 'INJURIES_SUM'
    
    Replace NaN with 0
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
aggregates = calculate_group_aggregates(binned_df)

# check the data
#aggregates.head(15)

aggregates.sample(15)

The table should look like [this](assets/2.6_aggregate_1.png)

Just for fun, here's a plot of injuries in the North and South directions based on bin. This may also help you debug your code. Depending on how you defined your bins you may see slight differences. Here's what it might  [look like](assets/direction_injuries.png)

Note that as with 2.7 below, *we* extended the bin range a little bit beyond the max. You don't have to do this.

In [ ]:
def create_injuries_sum_chart_vis(visdf):
    return alt.Chart(visdf).mark_point().encode(
        alt.Color('STREET_DIRECTION'),
        alt.X('BIN'),
        alt.Y('INJURIES_SUM')
    )

create_injuries_sum_chart_vis(aggregates)

In [ ]:
# we can also look at the differences between injuries and accidents for a direction. We can plot
# both directions so you can see the difference

def create_injuries_vs_accident_vis(visdf,chart_title):
    c1 = alt.Chart(visdf).mark_line().encode(
        alt.X('BIN'),
        alt.Y('INJURIES_SUM',scale=alt.Scale(domain=(0, 170)), title='Inj. (B)')
    )
    
    c2 = c1.mark_line(color='red').encode(
        alt.Y('ACCIDENT_COUNT',scale=alt.Scale(domain=(0, 170)), title='Acc. (R)')
    )
    return (c1+c2).properties(title=chart_title,height=100)

def create_compound_i_vs_a_vis(visdf):
    north = create_injuries_vs_accident_vis(aggregates[aggregates.STREET_DIRECTION == "N"],"North")
    south = create_injuries_vs_accident_vis(aggregates[aggregates.STREET_DIRECTION == "S"],"South")
    return north & south

create_compound_i_vs_a_vis(aggregates)


### 2.7 Sort the street ranges (10 points)

Because the street has both North and South addresses we are going to "stretch" it so the bins range from the highest North street value down to the lowest and then going from lowest South to highest South. Something like this (but we're going to used the binned values, instead of the "raw" house numbers, in the end):

!["Pulaski histogram model"](assets/pulaski_stretch.png)

Altair will use the sort order in the dataframe so if we sort the frame this way, that's what we'll have. 

- Sort the dataframe so North streets are in descending order and South streets are in ascending order
- You are provided with a 'pulaski_sort_order' arrray that contains this desired order. Use a categorical (pd.Categorial) column to order the dataframe according to this array.
- Note that as with 2.6, we are extending the bin range a little bit beyond the max (see the next cell). You don't have to do this.

In [ ]:
# pulaski_sort_order will be a useful way for you to bin
crashed_range = list(range(0, crashes_pulaski.STREET_NO.max()+1000, 300))
pulaski_sort_order = ['N ' + str(s) for s in crashed_range[::-1]] + ['S ' + str(s) for s in crashed_range]

In [ ]:
def categorical_sorting(df, sorder):
    """ Create a column called ORDER_LABEL that contains a concatenation of the street direction and the street range
    Set the sort order of this column to the provided sort array (sorder: the elements of this column should be in 
    the same order of the array, we can pass in pulaski_sort_order as below)
    Sort the dataframe (df) by this column
    The index of the data frame *SHOULD NOT* be modified
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
sorted_groups = categorical_sorting(aggregates, pulaski_sort_order)

# check the values
sorted_groups.sample(15)

The table should look like [this](assets/sorted_groups.png) though you might have some differences if you didn't use exactly the same bins we did.

You can test your code a few ways. First, we gave you the sort order, so you know what the ORDER_LABEL of the first row should be:

```assert sorted_groups['ORDER_LABEL'].iloc[0] == pulaski_sort_order[1]```

Note that you might have to change this assert. It might be pulaski_sort_order[0] with some other index (other than 1) depending on how you did the labeling.

You also know that the first item should be "greater" than the second, so you can test:

```assert sorted_groups['ORDER_LABEL'].iloc[0] > sorted_groups['ORDER_LABEL'].iloc[1]```

Finally, we indicated that your index should not be modified, thus (from the first row of our table):

```assert sorted_groups.loc[42]['ORDER_LABEL'] == "N 6300"```

Again, just for kicks, let's see where injuries happen. We're going to color bars by the bin and preserve our ascending/descending visualization. We can probably imagine other (better) ways to visualize this data, but this may be useful for you to debug. The visualization should look something like [this](assets/order_injuries.png)

If your X axis cutoffs are a bit different, that's fine. 

In [ ]:
def create_sorted_pulaski_histogram_vis(visframe,sorder):
    # creates a histogram based on the calculated values in visframe
    # assumes an ORDER_LABEL, INJURIES_SUM, and BIN columns
    # color will double encode the bin value (which is the X)
    return alt.Chart(visframe).mark_bar().encode(
        alt.X('ORDER_LABEL:O', sort=sorder),
        alt.Y('INJURIES_SUM:Q'),
        alt.Color('BIN:Q')
    ).properties(
        width=800
    )

create_sorted_pulaski_histogram_vis(sorted_groups,pulaski_sort_order)

Ok, let's actually make a useful visualization using some of the dataframes we've created. As a bonus, we're going to ask you what you would use this for.

In [ ]:
# to make the kind of chart we are interested in we're going to build it out of three different charts and
# put them together at the end

# this is going to be the left chart
bar_sorted_groups = sorted_groups[['ACCIDENT_COUNT','INJURIES_SUM']].unstack().reset_index() \
    .rename({'level_0':'TYPE','level_1':'SPEED',0:'COUNT'},axis=1)

# Note that we cheated a bit. The actual speed column (POSTED_SPEED) doesn't have enough variation for this
# example, so we're using the level_1 variable (it's an index variable) as a fake SPEED. 
# Just assume this actually is the speed at which the accident happened.

a = alt.Chart(bar_sorted_groups).mark_bar().transform_filter(alt.datum.TYPE == 'ACCIDENT_COUNT').encode(
    x=alt.X('COUNT:Q',sort='descending'),
    y=alt.Y('SPEED:O',axis=None),
    color=alt.Color('TYPE:N', 
                    legend=None,
                    scale=alt.Scale(domain=['ACCIDENT_COUNT', 'INJURIES_SUM'],
                                    range=['blue', 'orange']))
).properties(
    title='ACCIDENT_COUNT',
    width=300,
    height=900
)

# middle "chart" which actually won't be a chart, just a bunch of labels
b = alt.Chart(bar_sorted_groups).mark_bar().transform_filter(alt.datum.TYPE == 'ACCIDENT_COUNT').encode(
    y=alt.Y('SPEED:O', axis=None),
    text=alt.Text('SPEED:Q')
).mark_text().properties(title='SPEED',
                         width=20,
                         height=900)

# and the right most chart
c = alt.Chart(bar_sorted_groups).mark_bar().transform_filter(alt.datum.TYPE == 'INJURIES_SUM').encode(
    x='COUNT:Q',
    y=alt.Y('SPEED:O',axis=None),
    color=alt.Color('TYPE:N', 
                    legend=None,
                    scale=alt.Scale(domain=['ACCIDENT_COUNT', 'INJURIES_SUM'],
                                    range=['blue', 'orange']))
).properties(
    title='INJURIES_SUM',
    width=300,
    height=900
)

# put them all together 

a | b | c


## 2.8 (Bonus) Accident barchart visualization (up to 2 points)
Looking at the visualization we generated above (part 2.7), what domain/abstract tasks are fulfilled by this visualization? List at least one domain task and the corresponding abstract task. See the comment in the code about "speed."

_2.8 Answer_